The information about the data can be viewed here： https://www.kaggle.com/competitions/digit-recognizer/data

Firstly, import library:

In [ ]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from tensorflow import keras

Then load data 

In [ ]:
train_pd = pd.read_csv('../input/digit-recognizer/train.csv')
test_pd = pd.read_csv('../input/digit-recognizer/test.csv')

train_x = train_pd.drop('label', axis=1).to_numpy() / 255.0
train_x = train_x.reshape(-1, 28, 28, 1)


train_y = train_pd['label'].to_numpy()
print(train_x.shape)
print(train_y.shape)

plt.imshow(train_x[765].reshape(28,28))
plt.show()
print(train_y[765])

test_x = test_pd.to_numpy() / 255.0
test_x = test_x.reshape(-1, 28, 28, 1)

create NN model

In [ ]:
# method 1:

# model = keras.Sequential([
#     keras.layers.Conv2D(input_shape=(28, 28, 1), kernel_size=(3, 3), filters=32, padding='same', activation='relu', name="conv1_1"),
#     keras.layers.Conv2D(kernel_size=(3, 3), filters=64, padding='same', activation='relu', name="conv1_2"),
#     keras.layers.MaxPool2D(pool_size=(2, 2)),
#     keras.layers.Dropout(0.3),
#     keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding='same', activation='relu'),
#     keras.layers.Conv2D(kernel_size=(3, 3), filters=256, padding='valid', activation='relu'),
#     keras.layers.MaxPool2D(pool_size=(2, 2)),
#     keras.layers.Dropout(0.3),
#     keras.layers.Flatten(),
#     keras.layers.Dense(512, activation='relu'),
#     keras.layers.Dropout(0.3),
#     keras.layers.Dense(256, activation='relu'),
#     keras.layers.Dropout(0.3),
#     keras.layers.Dense(10, activation='softmax')
# ])

# model.compile(optimizer=keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['acc'])

# model.summary()


#Method 2 :

# num_classes = 2

# model = Sequential()
# model.add(Conv2D(64, (3, 3), strides=(1, 1), input_shape=(32, 32, 3), padding='same', activation='relu',
#                  kernel_initializer='uniform'))
# model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(128, (3, 2), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))


# Method 3: 
class VGG13_Model(tf.keras.Model):
    def __init__(self):
        super(VGG13_Model, self).__init__()

        layers_conv = [Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.relu),  # [b,32,32,3]
                       Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.relu),
                       MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),  # shape: [b,16,16,64]

                       Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.relu),
                       Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.relu),
                       MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),  # shape: [b,8,8,128]

                       Conv2D(filters=256, kernel_size=3, padding='same', activation=tf.nn.relu),
                       Conv2D(filters=256, kernel_size=3, padding='same', activation=tf.nn.relu),
                       MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),  # shape: [b,4,4,256]

                       Conv2D(filters=512, kernel_size=3, padding='same', activation=tf.nn.relu),
                       Conv2D(filters=512, kernel_size=3, padding='same', activation=tf.nn.relu),
                       MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),  # shape: [b,2,2,512]

                       Conv2D(filters=512, kernel_size=3, padding='same', activation=tf.nn.relu),
                       Conv2D(filters=512, kernel_size=3, padding='same', activation=tf.nn.relu),
                       MaxPool2D(pool_size=[2, 2], strides=2, padding='same')]  # shape: [b,1,1,512]
        self.seq_conv = tf.keras.Sequential(layers=layers_conv)
        layers_fc = [Dense(units=256, activation=tf.nn.relu),  # shape: [b,256]
                     Dense(units=128, activation=tf.nn.relu),  # shape: [b,256]
                     Dense(units=100, activation=None)]  # shape: [b,100]
        self.seq_fc = tf.keras.Sequential(layers_fc)


train data & show result

In [ ]:
hist = model.fit(train_x, train_y, batch_size=6720, epochs=30, validation_split=0.2)
plt.plot(hist.history['loss'], 'b-', label='loss')
plt.plot(hist.history['val_loss'], 'r--', label='val_loss')
plt.show()

plt.plot(hist.history['acc'], 'b-', label='acc')
plt.plot(hist.history['val_acc'], 'r--', label='val_acc')
plt.show()

In [ ]:
results = model.predict(test_x)

results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("vgg_mnist_submission.csv",index=False)